<a href="https://colab.research.google.com/github/umam1n/pbo/blob/main/capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

tickers = ["BBRI.JK", "TLKM.JK", "ANTM.JK", "ASII.JK"]
df = pd.DataFrame()

for ticker in tickers:
       try:
           data = yf.download(ticker, start="2021-01-01", end="2025-05-01", auto_adjust=False)
           df[ticker] = data['Close']  # Store closing prices for each ticker
           time.sleep(5)  # Wait for 5 seconds before the next request
       except yf.YFRateLimitError:
           print(f"Rate limited for {ticker}. Retrying in 60 seconds...")
           time.sleep(60)  # Wait for 60 seconds and try again
           data = yf.download(ticker, start="2021-01-01", end="2025-04-01", auto_adjust=False)
           df[ticker] = data['Close']

# Check if data is empty for any stock
if df.empty:
    raise ValueError(f"No data found for {tickers}. Check ticker symbols or try different stocks.")

# Extract closing prices for each stock
df_close = df['Close'].dropna()  # Drop missing values

# Normalize data for LSTM training
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(df_close)

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BBRI.JK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TLKM.JK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ANTM.JK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ASII.JK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


ValueError: No data found for ['BBRI.JK', 'TLKM.JK', 'ANTM.JK', 'ASII.JK']. Check ticker symbols or try different stocks.

In [ ]:
# Create sequences for LSTM training
def create_sequences(data, time_steps=50):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 50
X, y = create_sequences(data_scaled, time_steps)

# Reshape data for LSTM input
X = np.reshape(X, (X.shape[0], X.shape[1], len(tickers)))  # Multiple stocks as features

In [ ]:
# Split data into training (80%) and testing (20%)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Define LSTM model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(time_steps, len(tickers))),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=25),
    Dense(units=len(tickers))  # One output per stock
])

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32)


In [ ]:
# Predict on test data
y_pred = model.predict(X_test)

# Inverse transform predictions to original scale
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

# Plot actual vs predicted prices for each stock
plt.figure(figsize=(14, 7))
for i, ticker in enumerate(tickers):
    plt.plot(df_close.index[-len(y_test):], y_test_inv[:, i], label=f'Actual {ticker}', linestyle='dotted')
    plt.plot(df_close.index[-len(y_pred):], y_pred_inv[:, i], label=f'Predicted {ticker}')
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.legend()
plt.show()

In [ ]:
# Get last known stock prices to start forecasting
future_input = data_scaled[-time_steps:]  # Last 50 days
future_input = np.reshape(future_input, (1, time_steps, len(tickers)))  # Reshape for LSTM

# Predict next 100 days
future_predictions = []
for _ in range(100):  # Predict for 100 future days
    predicted_stock = model.predict(future_input)
    future_predictions.append(predicted_stock[0])

    # Append new prediction and remove oldest input to maintain sequence length
    future_input = np.append(future_input[:, 1:, :], [[predicted_stock[0]]], axis=1)

# Convert predictions back to original scale
future_predictions = scaler.inverse_transform(np.array(future_predictions))

# Generate future dates
future_dates = pd.date_range(start=df_close.index[-1], periods=100, freq='B')  # Business days

# Plot future predictions
plt.figure(figsize=(14, 7))
for i, ticker in enumerate(tickers):
    plt.plot(future_dates, future_predictions[:, i], label=f'Predicted {ticker}', linestyle='dashed')

plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.legend()
plt.title("Predicted Stock Prices for Next 100 Days")
plt.show()
